

## ========== Ignorando Filtros ========== 

Cenário:
No relatório tem 4 Segmento de dados,e a ideia é comparar o Cenário 1 x Cenário 2
 - Cenário 1 
 - Período 1
 - Cenário 2 
 - Período 2
 

<br>
<b>1)</b> Criar uma tabela chamada d_PERIODO_1 e uma outra identica chamada d_PERIODO2

<b>2)</b> Relacionar d_PERIODO_1 e d_PERIODO_2 com a d_CALENDARIO (usando ano.mes)

<b>3)</b> Criar uma d_CALENDARIO (precisa ter a coluna ano.mes)

<b>4)</b> criar d_CENARIO, d_CANARIO_1, d_CENARIO_2 e fazer os relarionamentos 

###### <i>Todas essas Tabelas e Relacionamentos estão abaixo, com os códigos TMDL </i>

## ================================

In [ ]:
// Para usar o filtro vc deve usar a tabela e coluna, exemplo
 CALCULATE(
    SUM(f_TABELA[Valor]),
    f_TABELA[COLUNA] = "X")

// para remover filtro, só usa a tabela
 CALCULATE(
    SUM(f_TABELA[Valor]),
    REMOVEFILTERS('d_DIMENSAO')), 

In [ ]:
//Para criar uma medida que ignore o cenário 1 e periodo 1 ou vice e versa, devemos usar o REMOVEFILTERS

VALOR_1 = 
    CALCULATE(
        SUM('f_DADOS'[Valor]),
        'f_DADOS'[Indicador] = "aqui seria um filtro",
        REMOVEFILTERS('d_PERIODO_2'), 
        REMOVEFILTERS(d_VERSAO_2)
    )


In [ ]:
//Para criar uma medida que ignore o cenário 2 e periodo 2 ou vice e versa, devemos usar o REMOVEFILTERS

VALOR_2 = 
    CALCULATE(
        SUM('f_DADOS'[Valor]),
        'f_DADOS'[Indicador] = "aqui seria um filtro",
        REMOVEFILTERS('d_PERIODO_1'), 
        REMOVEFILTERS(d_VERSAO_1)
    )

 ## TMDL

In [ ]:

createOrReplace

	table d_PERIODO_1
		isHidden
		lineageTag: fe92d078-2202-4c7c-ad34-e74b08318e24

		column 'ano.mes'
			isHidden
			lineageTag: 6eb85c7c-5508-4637-86d7-1fecbd1e130f
			summarizeBy: none
			isNameInferred
			sourceColumn: [ano.mes]

			annotation SummarizationSetBy = Automatic

		column Seleção
			isHidden
			lineageTag: 02a4dc79-131c-444d-951e-c5744e12f020
			summarizeBy: none
			isNameInferred
			sourceColumn: [Seleção]
			sortByColumn: Ordem

			annotation SummarizationSetBy = Automatic

		column Grupamento
			isHidden
			lineageTag: 5a3e1bc2-8c63-446e-ae0f-2d3f25c9b300
			summarizeBy: none
			isNameInferred
			sourceColumn: [Grupamento]

			annotation SummarizationSetBy = Automatic

		column Ordem
			isHidden
			formatString: 0
			lineageTag: 941f7e88-2638-44ca-a210-a8c894d428c7
			summarizeBy: sum
			isNameInferred
			sourceColumn: [Ordem]

			annotation SummarizationSetBy = Automatic

		partition d_PERIODO_1 = calculated
			mode: import
			source = ```
					
					VAR tbl =
					SUMMARIZE(
					        d_CALENDARIO,
					        d_CALENDARIO[ano.mes],
					        d_CALENDARIO[Ano],
					        d_CALENDARIO[Mês],
					        d_CALENDARIO[Trimestre_ANO],
					        d_CALENDARIO[Trimestre_NUM]
					    )
					VAR Meses = 
					SELECTCOLUMNS(
					    tbl,
					    "ano.mes", d_CALENDARIO[ano.mes],
					    "Seleção", FORMAT(DATE(d_CALENDARIO[Ano],d_CALENDARIO[Mês],1),"YY-MMM"),
					    "Grupamento", "Mensal",
					    "Ordem", 100*d_CALENDARIO[Ano] + d_CALENDARIO[Mês]
					)
					VAR Trimestres = 
					SELECTCOLUMNS(
					    tbl,
					    "ano.mes", d_CALENDARIO[ano.mes],
					    "Seleção", d_CALENDARIO[Trimestre_ANO],
					    "Grupamento", "Trimestral",
					    "Ordem", 100*d_CALENDARIO[Ano] + 10 * d_CALENDARIO[Trimestre_NUM] + 10
					)
					VAR Anos =
					SELECTCOLUMNS(
					    tbl,
					    "ano.mes", d_CALENDARIO[ano.mes],
					    "Seleção", d_CALENDARIO[Ano],
					    "Grupamento", "Anual",
					    "Ordem", 100*d_CALENDARIO[Ano] 
					)
					
					VAR YTD_base =
					GENERATE(
					        tbl,
					        VAR ano = d_CALENDARIO[Ano]
					        VAR mes = d_CALENDARIO[Mês]
					        RETURN
					        CALCULATETABLE(
					            SELECTCOLUMNS(
					                SUMMARIZE(
					                    d_CALENDARIO,
					                    d_CALENDARIO[Ano],
					                    d_CALENDARIO[Mês]
					                ),
					                "@Selecao", FORMAT(DATE(d_CALENDARIO[Ano], d_CALENDARIO[Mês],1),"M\MYY"),
					                "@Ordem", 1000*d_CALENDARIO[Ano] + d_CALENDARIO[Mês]
					            ),
					            REMOVEFILTERS(),
					            d_CALENDARIO[Ano]=ano,
					            d_CALENDARIO[Mês]>=mes            
					        )
					    )
					VAR YTD_Final =
					SELECTCOLUMNS(
					    YTD_base,
					    "ano.mes", d_CALENDARIO[ano.mes],
					    "Seleção", [@Selecao],
					    "Grupamento", "YTD",
					    "Ordem", [@Ordem]
					)
					
					
					VAR Resultado = 
					UNION(
					    Meses,
					    Trimestres,
					    Anos,
					    YTD_Final
					)
					RETURN Resultado
					```

		annotation PBI_Id = 8ff5597298064c4cad0a43a70a84441c



In [ ]:
createOrReplace

	table d_PERIODO_2
		isHidden
		lineageTag: eda0ebaf-ce28-4ffb-a35e-11fe493e796e

		measure 'Seleção Periodo 2' =
				
				SELECTEDVALUE('d_PERIODO_2'[Seleção])
			isHidden
			lineageTag: fbe0cf8c-1730-42b2-8860-27dfc0f73433

		column 'ano.mes'
			isHidden
			lineageTag: e2604434-7702-4ec2-8fde-5da88fa00153
			summarizeBy: none
			isNameInferred
			sourceColumn: d_PERIODO_1[ano.mes]
			columnOrigin: d_PERIODO_1.'ano.mes'

			annotation SummarizationSetBy = Automatic

		column Seleção
			isHidden
			lineageTag: b43f0354-cfc2-407d-a81a-5f6716fc166f
			summarizeBy: none
			isNameInferred
			sourceColumn: d_PERIODO_1[Seleção]
			sortByColumn: Ordem
			columnOrigin: d_PERIODO_1.Seleção

			annotation SummarizationSetBy = Automatic

		column Grupamento
			isHidden
			lineageTag: cc777446-e20e-4884-961c-2267d0b45a98
			summarizeBy: none
			isNameInferred
			sourceColumn: d_PERIODO_1[Grupamento]
			columnOrigin: d_PERIODO_1.Grupamento

			annotation SummarizationSetBy = Automatic

		column Ordem
			isHidden
			formatString: 0
			lineageTag: e1bd5cf4-fbc6-4306-82ea-d8c52cb9e109
			summarizeBy: sum
			isNameInferred
			sourceColumn: d_PERIODO_1[Ordem]
			columnOrigin: d_PERIODO_1.Ordem

			annotation SummarizationSetBy = Automatic

		partition d_PERIODO_2 = calculated
			mode: import
			source =
					
					'd_PERIODO_1'

		annotation PBI_Id = 1ae7dcacfb50445d8ea02c2b87d68087



In [ ]:
createOrReplace

	table d_CALENDARIO
		isHidden
		lineageTag: 223adca2-c43e-41eb-a8f5-e4419526822e

		column Data
			isHidden
			formatString: Short Date
			lineageTag: 4ec8d641-1346-461a-a481-5d2cd83e03ae
			summarizeBy: none
			sourceColumn: [Date]

			annotation SummarizationSetBy = Automatic

			annotation UnderlyingDateTimeDataType = Date

		column Ano = YEAR(d_CALENDARIO[Data])
			isHidden
			formatString: 0
			lineageTag: c200b85c-06b5-47d9-8424-02563d006df9
			summarizeBy: sum

			annotation SummarizationSetBy = Automatic

		column Mês = MONTH(d_CALENDARIO[Data])
			isHidden
			formatString: 0
			lineageTag: b9fed901-7e88-4724-b1ed-11995f6acd8f
			summarizeBy: none

			annotation SummarizationSetBy = User

		column Trimestre =
				
				FORMAT(
				d_CALENDARIO[Data], "QT")
			isHidden
			lineageTag: 3dddf301-e596-42e9-928e-f32fb70de54d
			summarizeBy: none

			annotation SummarizationSetBy = Automatic

		column Trimestre_NUM = YEAR(d_CALENDARIO[Data])*10 + QUARTER(d_CALENDARIO[Data])
			isHidden
			formatString: 0
			lineageTag: bd0d8668-a2b0-437a-8601-703b2d10292b
			summarizeBy: sum

			annotation SummarizationSetBy = Automatic

		column trimano = d_CALENDARIO[Trimestre_NUM]&"T"& RIGHT(d_CALENDARIO[Ano],2)
			isHidden
			lineageTag: d1758d42-ac2d-499e-be9b-d2fb7bd0ff06
			summarizeBy: none

			annotation SummarizationSetBy = Automatic

		column 'ano.mes' =
				
				VAR MES_ = IF(d_CALENDARIO[Mês] < 10, CONCATENATE("0",d_CALENDARIO[Mês]),d_CALENDARIO[Mês])
				
				RETURN
				FORMAT([Data],"YYYY.MM") --d_CALENDARIO[Ano]&"."& MES_
			isHidden
			lineageTag: 5e4b8779-cb70-4f98-9c34-0bd49a6446ae
			summarizeBy: none
			sortByColumn: 'Ano.MesNum'

			annotation SummarizationSetBy = Automatic

		column 'Ano.MesNum' = 12* YEAR([Data]) + MONTH([Data]) - 1
			isHidden
			formatString: 0
			lineageTag: c4e635a2-b370-4698-bcb2-04edf67529f7
			summarizeBy: sum

			annotation SummarizationSetBy = Automatic

		column Mes_Nome =
				
				UPPER(LEFT(FORMAT(d_CALENDARIO[Data], "mmm"), 1)) &
				LOWER(RIGHT(FORMAT(d_CALENDARIO[Data], "mmm"), LEN(FORMAT(d_CALENDARIO[Data], "mmm")) - 1))
			isHidden
			lineageTag: c0ca7d6d-e9d1-4a15-9b82-6732c04da249
			summarizeBy: none
			sortByColumn: Mês

			annotation SummarizationSetBy = Automatic

		column Trimestre_ANO =
				
				FORMAT(
				d_CALENDARIO[Data], "QTYY")
			isHidden
			lineageTag: 0cfb4fce-eb56-4473-a880-d16489811516
			summarizeBy: none
			sortByColumn: Ano

			annotation SummarizationSetBy = Automatic

		hierarchy 'Data Hierarquia'
			isHidden
			lineageTag: d941bd35-1f18-41fd-b9d5-49c00364aed8

			level Data
				lineageTag: fe4188e3-5c46-4927-8375-45b6be1a2806
				column: Data

		partition d_CALENDARIO = calculated
			mode: import
			source = CALENDARAUTO()

		annotation PBI_Id = 9908287fee44457993013b018046abb2



In [ ]:
createOrReplace

	relationship 5da4c3d0-c9f3-d6ad-1385-1f31882d5a50
		toCardinality: many
		fromColumn: d_CALENDARIO.'ano.mes'
		toColumn: d_PERIODO_1.'ano.mes'



In [ ]:
createOrReplace

	relationship d9189b96-eee3-fd8c-8b69-04b52d9261dc
		toCardinality: many
		fromColumn: d_CALENDARIO.'ano.mes'
		toColumn: d_PERIODO_2.'ano.mes'



In [ ]:
createOrReplace

	table d_VERSAO
		isHidden
		lineageTag: 1df7aa02-c95f-4a4b-be4a-8fb1d32346dd

		measure 'Nome Arquivo' = SELECTEDVALUE(d_VERSAO[Nome_Arquivo])
			isHidden
			lineageTag: 8b2ca34c-6b40-42a5-8f8c-75d8961ce7c7

			changedProperty = IsHidden

			annotation PBI_FormatHint = {"isGeneralNumber":true}

		column id_arquivo
			dataType: string
			isHidden
			lineageTag: dbb41c79-7e60-49fd-8718-bc1f9836cd1e
			summarizeBy: none
			sourceColumn: id_arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column Nome_Arquivo
			dataType: string
			isHidden
			lineageTag: 02beb194-e5f8-4ab1-8a75-f3c0b787be6a
			summarizeBy: none
			sourceColumn: Nome_Arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column Arquivo
			dataType: string
			isHidden
			lineageTag: fee5866d-a665-49e8-9c6e-cc21291ba862
			summarizeBy: none
			sourceColumn: Arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column Mes_OL = ```
				
				IF(
				    LEFT(d_VERSAO[Nome_Arquivo], 2) = "PE", 
				    0, 
				    VALUE(MID(d_VERSAO[Nome_Arquivo], 10, 2))
				)
				
				```
			lineageTag: 0a972274-4c0d-4ef5-85e8-72d89ef99c23
			summarizeBy: sum

			annotation SummarizationSetBy = Automatic

			annotation PBI_FormatHint = {"isGeneralNumber":true}

		partition d_VERSAO = m
			mode: import
			queryGroup: DIMENSÃO
			source =
					let
					    Fonte = f_DADOS,
					    #"Outras Colunas Removidas" = Table.SelectColumns(Fonte,{"Arquivo"}),
					    #"Duplicatas Removidas" = Table.Distinct(#"Outras Colunas Removidas"),
					    #"Consultas Mescladas" = Table.NestedJoin(#"Duplicatas Removidas", {"Arquivo"}, d_OL, {"Arquivo"}, "d_OL", JoinKind.LeftOuter),
					    #"d_OL Expandido" = Table.ExpandTableColumn(#"Consultas Mescladas", "d_OL", {"id_arquivo_OL"}, {"d_OL.id_arquivo_OL"}),
					    #"Consultas Mescladas1" = Table.NestedJoin(#"d_OL Expandido", {"Arquivo"}, d_PE, {"Arquivo"}, "d_PE", JoinKind.LeftOuter),
					    #"d_PE Expandido" = Table.ExpandTableColumn(#"Consultas Mescladas1", "d_PE", {"id_arquivo_PE"}, {"d_PE.id_arquivo_PE"}),
					    #"Colunas Mescladas" = Table.CombineColumns(#"d_PE Expandido",{"d_OL.id_arquivo_OL", "d_PE.id_arquivo_PE"},Combiner.CombineTextByDelimiter("", QuoteStyle.None),"id_arquivo"),
					    #"Texto Inserido Antes do Delimitador" = Table.AddColumn(#"Colunas Mescladas", "Nome_Arquivo", each Text.BeforeDelimiter([Arquivo], "."), type text)
					in
					    #"Texto Inserido Antes do Delimitador"

		changedProperty = IsHidden

		annotation PBI_NavigationStepName = Navegação

		annotation PBI_ResultType = Table



In [ ]:
createOrReplace

	table d_VERSAO_1
		isHidden
		lineageTag: c177685e-6641-494f-8401-eec17eae8381

		column versao_arquivo
			dataType: string
			isHidden
			lineageTag: 05a6ed72-58f7-4a56-be15-d5cf312b2e94
			summarizeBy: none
			sourceColumn: versao_arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column cenario
			dataType: string
			isHidden
			lineageTag: 46516448-8758-42ca-b305-d6046db14859
			summarizeBy: none
			sourceColumn: cenario

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column legenda_grafico
			dataType: string
			isHidden
			lineageTag: feb98110-6c4b-4096-aa1c-455bca347219
			summarizeBy: none
			sourceColumn: legenda_grafico

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column id_arquivo
			dataType: string
			isHidden
			lineageTag: 428213e2-a3ec-46f6-8424-e94965b76a5b
			summarizeBy: none
			sourceColumn: id_arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column Nome_Arquivo
			dataType: string
			isHidden
			lineageTag: 396453d6-4758-45c5-a442-d7fdcffa9edc
			summarizeBy: none
			sourceColumn: Nome_Arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column Arquivo
			dataType: string
			isHidden
			lineageTag: b86bc367-8407-476f-9a00-dedc016c28bb
			summarizeBy: none
			sourceColumn: Arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		partition d_VERSAO_1 = m
			mode: import
			queryGroup: DIMENSÃO
			source =
					let
					    Fonte = d_VERSAO,
					    #"Personalização Adicionada" = Table.AddColumn(Fonte, "versao_arquivo", each if (Text.Contains([Nome_Arquivo],"Outlook"))
					then Text.Middle([Nome_Arquivo],7,8)
					else if (Text.Contains([Nome_Arquivo],"PE"))
					then Text.Middle([Nome_Arquivo],2,8)
					else ""),
					    #"Texto Inserido Antes do Delimitador" = Table.AddColumn(#"Personalização Adicionada", "cenario", each Text.BeforeDelimiter([Arquivo], "2"), type text),
					    #"Coluna Duplicada" = Table.DuplicateColumn(#"Texto Inserido Antes do Delimitador", "Arquivo", "legenda_grafico"),
					    #"Texto Extraído Antes do Delimitador" = Table.TransformColumns(#"Coluna Duplicada", {{"legenda_grafico", each Text.BeforeDelimiter(_, "v"), type text}}),
					    #"Valor Substituído1" = Table.ReplaceValue(#"Texto Extraído Antes do Delimitador","Outlook","OL ",Replacer.ReplaceText,{"legenda_grafico"}),
					    #"Linhas Classificadas" = Table.Sort(#"Valor Substituído1",{{"Arquivo", Order.Descending}})
					in
					    #"Linhas Classificadas"

		changedProperty = IsHidden

		annotation PBI_NavigationStepName = Navegação

		annotation PBI_ResultType = Table



In [ ]:
createOrReplace

	table d_VERSAO_2
		isHidden
		lineageTag: 1908872c-32fc-49df-a11e-36488f365919

		column versao_arquivo
			dataType: string
			isHidden
			lineageTag: 980ddd11-4955-4ede-988b-1d7bc9c64f8a
			summarizeBy: none
			sourceColumn: versao_arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column cenario
			dataType: string
			isHidden
			lineageTag: 0a3c207b-c194-470f-bf63-9c9a96d30726
			summarizeBy: none
			sourceColumn: cenario

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column legenda_grafico
			dataType: string
			isHidden
			lineageTag: 8e723b09-ddd7-490a-84e7-99ae10c116a6
			summarizeBy: none
			sourceColumn: legenda_grafico

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column id_arquivo
			dataType: string
			isHidden
			lineageTag: c29ae2da-db4f-4abd-a281-524561d1a0ee
			summarizeBy: none
			sourceColumn: id_arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column Nome_Arquivo
			dataType: string
			isHidden
			lineageTag: d717eb16-ee97-431e-8722-75097e0b8233
			summarizeBy: none
			sourceColumn: Nome_Arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		column Arquivo
			dataType: string
			isHidden
			lineageTag: 2a0ed8b8-7feb-4191-abfd-a3b8431e8c4c
			summarizeBy: none
			sourceColumn: Arquivo

			changedProperty = IsHidden

			annotation SummarizationSetBy = Automatic

		partition d_VERSAO_2 = m
			mode: import
			queryGroup: DIMENSÃO
			source =
					let
					    Fonte = d_VERSAO_1
					in
					    Fonte

		changedProperty = IsHidden

		annotation PBI_NavigationStepName = Navegação

		annotation PBI_ResultType = Exception



In [ ]:
createOrReplace

	relationship 53b6811c-15c5-cdc0-bef6-8580ad323f48
		toCardinality: many
		fromColumn: d_VERSAO.Arquivo
		toColumn: d_VERSAO_1.Arquivo



In [ ]:
createOrReplace

	relationship f4e1af6d-1c79-4c85-0e06-8bc4811cd354
		toCardinality: many
		fromColumn: d_VERSAO.Arquivo
		toColumn: d_VERSAO_2.Arquivo

